In [17]:
from pathlib import Path
import pandas as pd

In [20]:
# 批量所有样本做变异调用
fqdir = "/data/rawdata/BGI/iGT_drSNP_250324/20250321-T7S00934"
outdir = "/data/mengxf/Project/KML250324_drSNP_iGT/work/250324/result"
shell_dir = Path("../pipes").resolve()
shell_dir.mkdir(exist_ok=True)

samples = pd.read_excel("../../金墁利-A175V2-模拟测试.xlsx", usecols=['Sample'])['Sample'].unique().tolist()

with open(shell_dir / "all_pipe.sh", "w") as f:
    for samp in samples:
        sample_name = samp[-6:]
        fq1 = f"{fqdir}/{samp}_R1.fastq.gz"
        fq2 = f"{fqdir}/{samp}_R2.fastq.gz"

        with open("pipe_template.sh") as f1:
            content = f1.read()
        out_coutent = content.replace("{{tag_sample_name}}", sample_name).replace(
            "{{tag_fastq1}}", fq1).replace("{{tag_fastq2}}", fq2).replace("{{tag_outdir}}", outdir)

        with open(f"{shell_dir}/{sample_name}.sh", "w") as f2:
            f2.write(out_coutent)

        f.write(f"bash {shell_dir}/{sample_name}.sh &> {shell_dir}/{sample_name}.sh.log \n")

合并所有表格数据

In [23]:
sample_names = [samp[-6:] for samp in samples]
# 第一个作为初始数据框
sample_name = sample_names[0]
df = pd.read_csv(f"/data/mengxf/Project/KML250324_drSNP_iGT/work/250324/result/vcf/{sample_name}/{sample_name}.snp_anno.tsv", sep="\t")
df.rename(columns={"DP": "DP_"+sample_name, "AF": "AF_"+sample_name}, inplace=True)
for sample_name in sample_names[1:]:
    df2 = pd.read_csv(f"/data/mengxf/Project/KML250324_drSNP_iGT/work/250324/result/vcf/{sample_name}/{sample_name}.snp_anno.tsv", sep="\t")
    df2.rename(columns={"DP": "DP_"+sample_name, "AF": "AF_"+sample_name}, inplace=True)
    df = pd.merge(df, df2, on=["#RS_ID", "CHROM", "POS", "REF", "ALT"], how="outer")
# 输出
with pd.ExcelWriter("../result/all_samples_table.xlsx") as writer:
    df.to_excel(writer, sheet_name="all", index=False)
    # 输出删除 DP_ 的列
    df.drop(columns=[col for col in df.columns if col.startswith("DP_")], inplace=True)
    df.to_excel(writer, sheet_name="no-DP", index=False)